# Laboratorio 9
## Ataque y defensa de modelos de Deep Learning
Universidad del Valle de Guatemala<br>
Security Data Science<br>
Pablo Andrés Zamora Vásquez - 21780<br>
Diego Andrés Morales Aquino - 21762<br>

## Primera parte: Ataques

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Modelo entrenado
model = keras.models.load_model("malware_classification_model.keras")

# Recompilar con eager mode habilitado
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True  # Necesario para ART con TF2
)

c:\Users\Pablo Zamora\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
# Cargar datos
dataset_path = "malimg_paper_dataset_imgs"
img_height, img_width = 64, 64
batch_size = 32
seed = 42

datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.3)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=seed
)

X_test, y_test = next(val_generator)
for _ in range(len(val_generator) - 1):
    x, y = next(val_generator)
    X_test = np.concatenate((X_test, x))
    y_test = np.concatenate((y_test, y))

Found 2790 images belonging to 25 classes.


In [4]:
# Envolver modelo con ART

classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=25,
    input_shape=(64, 64, 3),
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    clip_values=(0.0, 1.0)
)

### Ataque 1: FGSM

In [ ]:
# Ataque FGSM
attack = FastGradientMethod(estimator=classifier, eps=0.1)
X_adv = attack.generate(x=X_test)

In [ ]:
# Evaluación
preds_original = np.argmax(classifier.predict(X_test), axis=1)
preds_adv = np.argmax(classifier.predict(X_adv), axis=1)
true_labels = np.argmax(y_test, axis=1)

print("Accuracy original:", accuracy_score(true_labels, preds_original))
print("Accuracy con ataque FGSM:", accuracy_score(true_labels, preds_adv))

Accuracy original: 0.9594982078853047
Accuracy con ataque FGSM: 0.23225806451612904
